In [ ]:
#Import the libraries

import numpy as np
import pandas as pd
from pathlib import Path
import itertools
from itertools import zip_longest
from collections import Counter
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from itertools import zip_longest

In [ ]:
# Load parquet
parquet_path = Path(
    r"C:Local_folder"
) / "dataset_risk_AKI_full_formatted_version10.parquet"

df_version10 = pd.read_parquet(parquet_path)

print(f"Loaded {len(df_version10):,} rows × {len(df_version10.columns)} columns")
df_version10.head()

### Transformations before to train the models

In [ ]:
# Count null values in each column
null_counts = df_version10.isnull().sum()

print(null_counts)

In [ ]:
df_version10 = df_version10.fillna(0)
df_version10.fillna(0, inplace=True)

In [ ]:
# Count null values in each column
null_counts = df_version10.isnull().sum()

print(null_counts)

In [ ]:

def mode_impute(series: pd.Series):
    """
    Impute the most frequent value in a Series using value_counts to avoid mixed-type sorting issues.
    Returns NaN if the series is empty.
    """
    try:
        return series.value_counts().idxmax()
    except Exception:
        return np.nan

# -----------------------------
# 1. Define column groups by aggregation rule (exclude 'meds_admin' here)
# -----------------------------
count_cols = [
    'n_diag_codes',
    'n_med_admin_events',
    'n_hcpcs_events',
    'n_icd_procs',
    'n_lab_events',
    'n_omr_events'
]
max_cols = ['BMI', 'Weight', 'BP']

numeric_cols = df_version10.select_dtypes(include=np.number).columns.tolist()
other_numeric = [
    col for col in numeric_cols
    if col not in count_cols + max_cols + ['subject_id']
]

cat_cols = [
    col for col in df_version10.select_dtypes(include=['object', 'category']).columns
    if col != 'meds_admin'
]

# -----------------------------
# 2. Build aggregation mapping (including meds_admin now by mode)
# -----------------------------
agg_dict = {}

for col in count_cols:
    agg_dict[col] = 'mean'

for col in max_cols:
    agg_dict[col] = 'max'

for col in other_numeric:
    agg_dict[col] = mode_impute

for col in cat_cols:
    agg_dict[col] = mode_impute

# Impute meds_admin by its mode per subject
agg_dict['meds_admin'] = mode_impute

# -----------------------------
# 3. Aggregate all columns in one go
# -----------------------------
df_unique = (
    df_version10
      .groupby('subject_id')
      .agg(agg_dict)
      .reset_index()
)

# df_unique now has one row per subject_id, with:
# - mean of count columns
# - max of BMI, Weight, BP
# - mode-imputed other numeric and categorical columns (including meds_admin)


In [ ]:
df_unique['outcome_aki'] = (df_unique['outcome_aki'] > 0).astype(int)
print(df_unique['outcome_aki'].unique())

counts = df_unique['outcome_aki'].value_counts().sort_index()
percentages = counts / counts.sum() * 100

fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.bar(counts.index.astype(str), counts.values, width=0.6)

for bar, count, pct in zip(bars, counts.values, percentages):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        count,
        f'{count}\n{pct:.1f}%',
        ha='center',
        va='bottom'
    )

ax.set_xlabel('outcome_aki')
ax.set_ylabel('Frecuencia')
ax.set_title('Frecuencia y porcentaje de outcome_aki')
plt.tight_layout()
plt.show()

In [ ]:
counts = {
    'BMI': (df_unique['BMI'] != 0).sum(),
    'Weight': (df_unique['Weight'] != 0).sum()
}

labels = list(counts.keys())
values = list(counts.values())

fig, ax = plt.subplots(figsize=(6, 4))
bars = ax.bar(labels, values)

for bar, value in zip(bars, values):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        value,
        f'{value}',
        ha='center',
        va='bottom'
    )

ax.set_title('Count of Non-Zero Records for BMI and Weight')
ax.set_xlabel('Variable')
ax.set_ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# We removed the column that has the least data from the previous graph, in this case BMI was removed, but it all depends on the previous step.
df_unique.drop(columns=['BMI'], inplace=True)

In [ ]:
output_path = Path(r"Local_carpet"

df_unique.to_csv(output_path, index=False)

print(f"Saved DataFrame to: {output_path}")

In [ ]:
###############################################################################################################################

In [ ]:
csv_path = Path(r"C:Local_carpet") / "df_unique.csv"

df_unique = pd.read_csv(csv_path)

df_unique.head()

### Transform and train models

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
df = df_unique.copy()

# Drop ID and unwanted columns
drop_cols = [c for c in df.columns if '_id' in c or c in ['n_diag_codes', 'diag_seq_nums', 'subject_id']]
df = df.drop(columns=drop_cols, errors='ignore')

# Separate features and target
X = df.drop(columns=['outcome_aki'], errors='ignore')
y = df['outcome_aki']

# Subsample 30% for quick turnaround
X_sub, _, y_sub, _ = train_test_split(X, y, train_size=0.3, stratify=y, random_state=42)

# Train/test split on subsample
X_train, X_test, y_train, y_test = train_test_split(
    X_sub, y_sub, test_size=0.3, stratify=y_sub, random_state=42
)

# Frequency-encode categorical columns
cat_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()
for col in cat_cols:
    freq = X_train[col].value_counts(normalize=True)
    X_train[col] = X_train[col].map(freq)
    X_test[col]  = X_test[col].map(freq).fillna(0)

# Scale numeric columns
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# Define lightweight models
models = {
    'LogisticRegression': LogisticRegression(max_iter=100, n_jobs=-1),
    'RandomForest':       RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1),
    'DecisionTree':       DecisionTreeClassifier(random_state=42),
    'XGBoost':            XGBClassifier(n_estimators=20, use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
}

# Train and evaluate
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }

# Display results
results_df = pd.DataFrame(results).T
print(results_df)

# Plot comparison
names = results_df.index.tolist()
accs  = results_df['Accuracy'].tolist()
f1s   = results_df['F1-Score'].tolist()
x = np.arange(len(names))
width = 0.35

fig, ax = plt.subplots(figsize=(8,5))
bars_acc = ax.bar(x - width/2, accs, width, label='Accuracy', color='skyblue')
bars_f1  = ax.bar(x + width/2, f1s,  width, label='F1-Score', color='salmon')
for bars in (bars_acc, bars_f1):
    for bar in bars:
        h = bar.get_height()
        ax.text(bar.get_x()+bar.get_width()/2, h+0.005, f'{h:.3f}', ha='center', va='bottom')

ax.set_xticks(x)
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_ylabel('Score')
ax.set_title('Quick Model Comparison')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
model_names = results_df.index.tolist()
acc_vals    = results_df['Accuracy'].tolist()
f1_vals     = results_df['F1-Score'].tolist()

x = np.arange(len(model_names))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 5))
bars_acc = ax.bar(x - width/2, acc_vals, width, label='Accuracy', color='skyblue')
bars_f1  = ax.bar(x + width/2, f1_vals,  width, label='F1-Score', color='salmon')

for bar in bars_acc:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.005, f'{height:.2f}', 
            ha='center', va='bottom', fontsize=9)

for bar in bars_f1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.005, f'{height:.2f}', 
            ha='center', va='bottom', fontsize=9)

ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.set_ylabel('Score')
ax.set_title('Model Comparison: Accuracy and F1-Score')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ————— Confusion Matrices —————
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    
    fig, ax = plt.subplots(figsize=(5, 5))
    disp.plot(ax=ax, cmap='Blues', colorbar=False)
    ax.set_title(f'Confusion Matrix — {model_name}')
    plt.tight_layout()
    plt.show()


In [ ]:
# ROC Curves

from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
for model_name, model in models.items():
    # Obtain scores for ROC
    if hasattr(model, "predict_proba"):
        y_score = model.predict_proba(X_test)[:, 1]
    else:
        y_score = model.decision_function(X_test)
    
    fpr, tpr, _ = roc_curve(y_test, y_score)
    auc = roc_auc_score(y_test, y_score)
    
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# important variables of each model

importancias_modelos = {}
for name, model in models.items():
    if hasattr(model, 'feature_importances_'):
        importancias_modelos[name] = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importancias_modelos[name] = np.abs(model.coef_[0])

feature_names = X_train.columns.tolist()

for name, importances in importancias_modelos.items():
    threshold = 0.01
    idxs = [i for i, val in enumerate(importances) if val > threshold]
    if len(idxs) == 0:
        continue
    vals = importances[idxs]
    names = [feature_names[i] for i in idxs]
    
    order = np.argsort(vals)[::-1]
    top_vals  = vals[order]
    top_names = [names[i] for i in order]
    
    fig, ax = plt.subplots(figsize=(12, max(6, len(top_vals) * 0.4)))
    ax.barh(range(len(top_vals)), top_vals, align='center')
    ax.set_yticks(range(len(top_vals)))
    ax.set_yticklabels(top_names, fontsize=10)
    ax.invert_yaxis()
    ax.set_title(f'Feature Importances — {name}')
    ax.set_xlabel('Relative Importance')
    plt.tight_layout()
    plt.show()